**DATA SCRAPING HnM CLOTHING - LADIES (27/06/2025)**

In [2]:
import requests
import json
import time
import pandas as pd

In [3]:
#Curl bash HnM 
headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8,id;q=0.7,ms;q=0.6',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    'Magento-Customer-Group': 'b6589fc6ab0dc82cf12099d1c2d40ab994e8410c',
    'Magento-Environment-Id': 'c9f844c8-6674-407c-a119-6cfb03575f4a',
    'Magento-Store-Code': 'hmid_store',
    'Magento-Store-View-Code': 'id_en',
    'Magento-Website-Code': 'hmid',
    'Origin': 'https://id.hm.com',
    'Pragma': 'no-cache',
    'Referer': 'https://id.hm.com/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
    'X-Api-Key': 'f0dd76b31bae40ebaa2819c0f25d6f2c',
    'X-Request-Id': '4ef10c7a-bf01-4fb2-bda9-6b359ac8edab', 
    'sec-ch-ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

# Base JSON data for the GraphQL query 
base_json_data = {
    'query': '\n    query productSearch(\n        $phrase: String!\n        $pageSize: Int\n        $currentPage: Int = 1\n        $filter: [SearchClauseInput!]\n        $sort: [ProductSearchSortInput!]\n        $context: QueryContextInput\n    ) {\n        productSearch(\n            phrase: $phrase\n            page_size: $pageSize\n            current_page: $currentPage\n            filter: $filter\n            sort: $sort\n            context: $context\n        ) {\n            total_count\n            items {\n                ...Product\n                ...ProductView\n            }\n            facets {\n                ...Facet\n            }\n            page_info {\n                current_page\n                page_size\n                total_pages\n            }\n        }\n        attributeMetadata {\n            sortable {\n                label\n                attribute\n                numeric\n            }\n        }\n    }\n    \n    fragment Product on ProductSearchItem {\n        product {\n            __typename\n            sku\n            id\n            uid\n            description {\n                html\n            }\n            short_description{\n                html\n            }\n            name\n            canonical_url\n            small_image {\n                url\n            }\n            image {\n                url\n            }\n            thumbnail {\n                url\n            }\n            price_range {\n                minimum_price {\n                    fixed_product_taxes {\n                        amount {\n                            value\n                            currency\n                        }\n                        label\n                    }\n                    regular_price {\n                        value\n                        currency\n                    }\n                    final_price {\n                        value\n                        currency\n                    }\n                    discount {\n                        percent_off\n                        amount_off\n                    }\n                }\n                maximum_price {\n                    fixed_product_taxes {\n                        amount {\n                            value\n                            currency\n                        }\n                        label\n                    }\n                    regular_price {\n                        value\n                        currency\n                    }\n                    final_price {\n                        value\n                        currency\n                    }\n                    discount {\n                        percent_off\n                        amount_off\n                    }\n                }\n            }\n        }\n    }\n\n    \n    fragment ProductView on ProductSearchItem {\n        productView {\n            __typename\n            sku\n            name\n            inStock\n            url\n            urlKey\n            images {\n                label\n                url\n                roles\n            }\n            links {\n                linkTypes\n                product {\n                    sku\n                    url\n                    inStock\n                    attributes {\n                        name\n                        value\n                    }\n                }\n            }\n            ... on ComplexProductView {\n                attributes {\n                    name\n                    value\n                }\n                priceRange {\n                    maximum {\n                        final {\n                            amount {\n                                value\n                                currency\n                            }\n                        }\n                        regular {\n                            amount {\n                                value\n                                currency\n                            }\n                        }\n                    }\n                    minimum {\n                        final {\n                            amount {\n                                value\n                                currency\n                            }\n                        }\n                        regular {\n                            amount {\n                                value\n                                currency\n                            }\n                        }\n                    }\n                }\n                options {\n                    id\n                    title\n                    values {\n                        title\n                        ... on ProductViewOptionValueSwatch {\n                            id\n                            inStock\n                            type\n                            value\n                        }\n                    }\n                }\n            }\n            ... on SimpleProductView {\n                price {\n                    final {\n                        amount {\n                            value\n                            currency\n                        }\n                    }\n                    regular {\n                        amount {\n                            value\n                            currency\n                        }\n                    }\n                }\n            }\n        }\n        highlights {\n            attribute\n            value\n            matched_words\n        }\n    }\n\n    \n    fragment Facet on Aggregation {\n        title\n        attribute\n        buckets {\n            title\n            __typename\n            ... on CategoryView {\n                name\n                count\n                path\n                level\n            }\n            ... on ScalarBucket {\n                id\n                count\n            }\n            ... on RangeBucket {\n                from\n                to\n                count\n            }\n            ... on StatsBucket {\n                min\n                max\n            }\n        }\n    }\n\n',
    'variables': {
        'phrase': '',
        'pageSize': 24,
        'currentPage': 1, # Start with page 1
        'filter': [
            {
                'attribute': 'categoryPath',
                'eq': 'ladies/clothing/ladies-view-all',
            },
            {
                'attribute': 'visibility',
                'in': [
                    'Catalog',
                    'Catalog, Search',
                ],
            },
            {
                'attribute': 'inStock',
                'eq': 'true',
            },
            {
                'attribute': 'price',
                'range': {
                    'from': 1,
                    'to': None,
                },
            },
        ],
        'sort': [
            {
                'attribute': 'position',
                'direction': 'ASC',
            },
        ],
        'context': {
            'customerGroup': 'b6589fc6ab0dc82cf12099d1c2d40ab994e8410c',
            'userViewHistory': [
                {
                    'sku': '1259420018',
                    'dateTime': '2025-06-03T04:35:17.590Z',
                },
                {
                    'sku': '1293967002',
                    'dateTime': '2025-06-03T04:36:05.172Z',
                },
            ],
        },
    },
}


In [9]:
all_products = []
current_page = 1
total_pages = 1 

while current_page <= total_pages:
    print(f"Scraping page {current_page}...")
    json_data = base_json_data.copy() # Create a copy of the JSON data
    json_data['variables']['currentPage'] = current_page

    try:
        response = requests.post('https://catalog-service.adobe.io/graphql', headers=headers, json=json_data)
        response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)
        data = response.json()

        # Extract page info only on the first request
        if current_page == 1:
            total_count = data['data']['productSearch']['total_count']
            total_pages = data['data']['productSearch']['page_info']['total_pages']
            print(f"Total products: {total_count}, Total pages: {total_pages}")

        products_on_page = data['data']['productSearch']['items']
        all_products.extend(products_on_page)

        current_page += 1
        time.sleep(1) # Be polite 
        
    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        break 
    except json.JSONDecodeError:
        print("Error decoding JSON response.")
        break
    except KeyError as e:
        print(f"Key error in response data: {e}. Response: {response.text}")
        break

# All products have been collected
print(f"Scraped {len(all_products)} products.")


Scraping page 1...
Total products: 4089, Total pages: 171
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping p

In [10]:
# Flatten the data and create a DataFrame
flattened_products = []
for item in all_products:
    product_info = {}
    product = item.get('product', {})
    product_view = item.get('productView', {})

    # Product details
    product_info['sku'] = product.get('sku') or product_view.get('sku')
    product_info['name'] = product.get('name') or product_view.get('name')
    product_info['canonical_url'] = product.get('canonical_url') or product_view.get('url')
    product_info['product_id'] = product.get('id')
    product_info['uid'] = product.get('uid')
    product_info['in_stock'] = product_view.get('inStock')

    # Descriptions
    product_info['description_html'] = product.get('description', {}).get('html')
    product_info['short_description_html'] = product.get('short_description', {}).get('html')

    # Image URLs
    image_urls = []
    if product.get('image', {}).get('url'):
        image_urls.append({'url': product['image']['url'], 'role': 'main_image'})
    if product.get('small_image', {}).get('url'):
        image_urls.append({'url': product['small_image']['url'], 'role': 'small_image'})
    if product.get('thumbnail', {}).get('url'):
        image_urls.append({'url': product['thumbnail']['url'], 'role': 'thumbnail'})
    
    if product_view.get('images'):
        for img in product_view['images']:
            image_urls.append({'url': img.get('url'), 'label': img.get('label'), 'roles': img.get('roles')})
    
    product_info['all_image_data'] = json.dumps(image_urls) # Store as JSON string if multiple

    # Price details 
    final_price_value = None
    final_price_currency = None
    regular_price_value = None
    regular_price_currency = None
    discount_percent_off = None
    discount_amount_off = None

    if product and product.get('price_range'):
        min_price = product['price_range'].get('minimum_price', {})
        if min_price.get('final_price'):
            final_price_value = min_price['final_price'].get('value')
            final_price_currency = min_price['final_price'].get('currency')
        if min_price.get('regular_price'):
            regular_price_value = min_price['regular_price'].get('value')
            regular_price_currency = min_price['regular_price'].get('currency')
        if min_price.get('discount'):
            discount_percent_off = min_price['discount'].get('percent_off')
            discount_amount_off = min_price['discount'].get('amount_off')
    elif product_view and product_view.get('price'): # SimpleProductView
        if product_view['price'].get('final'):
            final_price_value = product_view['price']['final']['amount'].get('value')
            final_price_currency = product_view['price']['final']['amount'].get('currency')
        if product_view['price'].get('regular'):
            regular_price_value = product_view['price']['regular']['amount'].get('value')
            regular_price_currency = product_view['price']['regular']['amount'].get('currency')
    elif product_view and product_view.get('priceRange'): # ComplexProductView
        min_range = product_view['priceRange'].get('minimum', {})
        if min_range.get('final'):
            final_price_value = min_range['final']['amount'].get('value')
            final_price_currency = min_range['final']['amount'].get('currency')
        if min_range.get('regular'):
            regular_price_value = min_range['regular']['amount'].get('value')
            regular_price_currency = min_range['regular']['amount'].get('currency')
            
    product_info['final_price_value'] = final_price_value
    product_info['final_price_currency'] = final_price_currency
    product_info['regular_price_value'] = regular_price_value
    product_info['regular_price_currency'] = regular_price_currency
    product_info['discount_percent_off'] = discount_percent_off
    product_info['discount_amount_off'] = discount_amount_off

    # Product Attributes (color, size)
    attributes = {}
    if product_view.get('attributes'):
        for attr in product_view['attributes']:
            attributes[attr.get('name')] = attr.get('value')
    product_info.update(attributes)    

    flattened_products.append(product_info)

df = pd.DataFrame(flattened_products)
print("Sample data:")
print(df.head())
print("DataFrame columns:")
print(df.columns)

# Save to CSV
df.to_csv('HnM_dataproducts_ladies_clothing_view_all.csv', index=False)
print("Data saved to HnM_dataproducts_ladies_clothing_view_all.csv")

Sample data:
          sku                       name  \
0  1297196002     Oversized poplin shirt   
1  1294598001      Wide High Ankle Jeans   
2  1260797016              Fine-knit top   
3  1289496001  Button-front sweater vest   
4  1298699001   Tailored denim waistcoat   

                                       canonical_url  product_id      uid  \
0  //id.hm.com/id_en/oversized-poplin-shirt-12971...     3516934  3516934   
1  //id.hm.com/id_en/wide-high-ankle-jeans-129459...     3453538  3453538   
2    //id.hm.com/id_en/fine-knit-top-1260797016.html     3509054  3509054   
3  //id.hm.com/id_en/button-front-sweater-vest-12...     3270216  3270216   
4  //id.hm.com/id_en/tailored-denim-waistcoat-129...     3481195  3481195   

   in_stock                                   description_html  \
0      True  Oversized shirt in crisp cotton poplin with a ...   
1      True  Jeans in cotton denim with a slight stretch fo...   
2      True  Short-sleeved top in a fine-knit viscose blend..